### 实例描述
#### 将代码“cifar卷积”中的代码使用tf.contrib.layers重构

In [1]:
from cifar10 import cifar10_input
import tensorflow as tf
import numpy as np

In [3]:
batch_size = 128
print("begin")
images_train, labels_train = cifar10_input.inputs(eval_data=False, batch_size=batch_size)
images_test, labels_test = cifar10_input.inputs(eval_data=True, batch_size=batch_size)
print("begin data")

begin
begin data


In [4]:
# 定义占位符
x = tf.placeholder(tf.float32, [None, 24, 24, 3])   # CIFAR数据集的shape 24*24*3
y = tf.placeholder(tf.float32, [None, 10])    # 0-9数字分类  -》 10类

x_image = tf.reshape(x, [-1, 24, 24, 3])

In [6]:
# tf.contrib.layers.conv2d,    tf.contrib.layers.max_pool2d
h_conv1 = tf.contrib.layers.conv2d(x_image, 64, 5, 1, 'SAME', activation_fn=tf.nn.relu)
h_pool1 = tf.contrib.layers.max_pool2d(h_conv1, [2, 2], stride=2, padding='SAME')

h_conv2 = tf.contrib.layers.conv2d(h_pool1, 64, [5, 5], 1, 'SAME', activation_fn=tf.nn.relu)
h_pool2 = tf.contrib.layers.max_pool2d(h_conv2, [2, 2], stride=2, padding='SAME')

nt_hpool2 = tf.contrib.layers.avg_pool2d(h_pool2, [6, 6], 6, padding='SAME')

nt_hpool2_flat = tf.reshape(nt_hpool2, [-1, 64])

# tf.contrib.layers.fully_connected
y_conv = tf.contrib.layers.fully_connected(nt_hpool2_flat, 10, activation_fn=tf.nn.softmax)

In [7]:
cross_entropy = -tf.reduce_sum(y * tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [9]:
sess = tf.Session(config=tf.ConfigProto(device_count={'gpu':0}))
sess.run(tf.global_variables_initializer())
for i in range(15000):  # 20000
    image_batch, label_batch = sess.run([images_train, labels_train])
    label_b = np.eye(10, dtype=float)[label_batch]    #one hot
    
    train_step.run(feed_dict={x:image_batch, y: label_b},session=sess)
    
    if i%200 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:image_batch, y: label_b},session=sess)
        print( "step %d, training accuracy %g"%(i, train_accuracy))

image_batch, label_batch = sess.run([images_test, labels_test])
label_b = np.eye(10,dtype=float)[label_batch]#one hot
print ("finished！ test accuracy %g"%accuracy.eval(feed_dict={x:image_batch, y: label_b},session=sess))

step 0, training accuracy 0.109375
step 200, training accuracy 0.28125
step 400, training accuracy 0.429688
step 600, training accuracy 0.429688
step 800, training accuracy 0.375
step 1000, training accuracy 0.390625
step 1200, training accuracy 0.375
step 1400, training accuracy 0.476562
step 1600, training accuracy 0.328125
step 1800, training accuracy 0.5
step 2000, training accuracy 0.46875
step 2200, training accuracy 0.445312
step 2400, training accuracy 0.523438
step 2600, training accuracy 0.53125
step 2800, training accuracy 0.421875
step 3000, training accuracy 0.40625
step 3200, training accuracy 0.546875
step 3400, training accuracy 0.570312
step 3600, training accuracy 0.523438
step 3800, training accuracy 0.507812
step 4000, training accuracy 0.507812
step 4200, training accuracy 0.460938
step 4400, training accuracy 0.460938
step 4600, training accuracy 0.523438
step 4800, training accuracy 0.617188
step 5000, training accuracy 0.539062
step 5200, training accuracy 0.460